In [1]:
import pandas as pd
import sqlite3
from decimal import Decimal
from enum import Enum
from prettytable import PrettyTable

To create based database run in terminal:
* `sqlite3 database.db`
* `DROP TABLE orders;` (if we want to clear database)
* `.read database.sql`
* `.schema` to check if table exist
* `SELECT * FROM orders;` to check based data

In [2]:
class OperationType(Enum):
    Add = "Add"
    Remove = "Remove"

In [3]:
class OrderType(Enum):
    Buy = "Buy"
    Sell = "Sell"

In [4]:
class Order:
    def __init__(self, id: str, order_type: OrderType, operation_type: OperationType, price: Decimal, quantity: int) -> None:
        self.id = id
        self.order_type = order_type # (Buy/Sell)
        self.operation_type = operation_type # (Add/Remove)
        self.price = price 
        self.quantity = quantity

In [5]:
class OrderBook:
    def __init__(self) -> None:
        self.con = sqlite3.connect('database.db')
        self.cur = self.con.cursor()
    
    def close_connection(self):
        self.con.close()

    def insert_order(self, order: Order):
        self.cur.execute('INSERT INTO "orders" ("id", "order", "type", "price", "quantity") VALUES (?, ?, ?, ?, ?)', [order.id, order.order_type.value, order.operation_type.value, str(order.price), order.quantity])
        self.con.commit()
    
    def find_order(self, id, operation_type:OperationType):
        self.cur.execute(""" SELECT * FROM "orders" WHERE "id" = ? AND "type" = ? """, [id, operation_type.value]) # assuming that there can be only one order
        rows = self.cur.fetchall()
        return rows[0]

    def best_buy(self):
        self.cur.execute(""" SELECT MAX(price) 
                        FROM "orders" 
                        WHERE id IN (
                            SELECT "id" 
                            FROM "orders" 
                            GROUP BY "id" 
                            HAVING COUNT(*) = 1
                        ) 
                        AND "order" = 'Buy'; """) 
        rows = self.cur.fetchall()
        return rows[0][0]

    def best_sell(self):
        self.cur.execute(""" SELECT MIN(price) 
                        FROM "orders" 
                        WHERE id IN (
                            SELECT "id" 
                            FROM "orders" 
                            GROUP BY "id" 
                            HAVING COUNT(*) = 1
                        ) 
                        AND "order" = 'Sell'; """) 
        rows = self.cur.fetchall()
        return rows[0][0]

    def add(self, order: Order):
        # check if id exist
        self.cur.execute(""" SELECT * FROM "orders" WHERE "id" = ? """, [order.id]) 
        rows = self.cur.fetchall()
        is_id = len(rows) > 0

        # if id not exist
        if not is_id:
            # insert new order
            self.insert_order(order=order)

            # print best price
            if order.order_type == OrderType.Buy:
                print(f"Aktualnie najlepsze cena zlecenia typu Sell: {self.best_sell()}")
            else:
                print(f"Aktualnie najlepsze cena zlecenia typu Buy: {self.best_buy()}")
        else:
            print(f"Zlecenie o id: {order.id} już istnieje")

    def remove(self, id):
        # check id exist
        self.cur.execute(""" SELECT * FROM "orders" WHERE "id" = ? """, [id]) 
        rows = self.cur.fetchall()
        is_id = len(rows) > 0

        # if order exist
        if is_id:
            # check that it has not been previously deleted
            self.cur.execute(""" SELECT * FROM "orders" WHERE "id" = ? AND type = 'Remove' """, [id]) # assuming that there can be only one order
            rows = self.cur.fetchall()
            is_remove = len(rows) > 0

            # if not
            if not is_remove:
                # find that order
                row = self.find_order(id=id, operation_type=OperationType.Add)

                # and remove it
                order = Order(id=id, order_type=OrderType[row[1]], operation_type=OperationType.Remove, price=Decimal(str(row[3])), quantity=row[4])
                self.insert_order(order=order)
            else:
                print(f"Id: {id} nie istnieje, lub zlecenie zostało już wycofane")
        else:
            print(f"Id: {id} nie istnieje, lub zlecenie zostało już wycofane")

    def print_orderbook(self):
        self.cur.execute("SELECT * FROM orders")
        rows = self.cur.fetchall()

        column_names = [description[0] for description in self.cur.description]
        
        table = PrettyTable()
        table.field_names = column_names
        
        for row in rows:
            table.add_row(row)

        print(table)
    


In [6]:
order_book = OrderBook()

In [7]:
order_book.print_orderbook()

+-----+-------+--------+-------+----------+
|  id | order |  type  | price | quantity |
+-----+-------+--------+-------+----------+
| 001 |  Buy  |  Add   |   20  |   100    |
| 002 |  Sell |  Add   |   25  |   200    |
| 003 |  Buy  |  Add   |   23  |    50    |
| 004 |  Buy  |  Add   |   23  |    70    |
| 003 |  Buy  | Remove |   23  |    50    |
| 005 |  Sell |  Add   |   28  |   100    |
+-----+-------+--------+-------+----------+


### Adding new order

Buy type

In [8]:
order = Order(id="006", order_type=OrderType.Buy, operation_type=OperationType.Add, price=Decimal("30.00"), quantity=30)
order_book.add(order=order)
order_book.print_orderbook()

Aktualnie najlepsze cena zlecenia typu Sell: 25
+-----+-------+--------+-------+----------+
|  id | order |  type  | price | quantity |
+-----+-------+--------+-------+----------+
| 001 |  Buy  |  Add   |   20  |   100    |
| 002 |  Sell |  Add   |   25  |   200    |
| 003 |  Buy  |  Add   |   23  |    50    |
| 004 |  Buy  |  Add   |   23  |    70    |
| 003 |  Buy  | Remove |   23  |    50    |
| 005 |  Sell |  Add   |   28  |   100    |
| 006 |  Buy  |  Add   |   30  |    30    |
+-----+-------+--------+-------+----------+


Sell type

In [9]:
order = Order(id="007", order_type=OrderType.Sell, operation_type=OperationType.Add, price=Decimal("27.00"), quantity=50)
order_book.add(order=order)
order_book.print_orderbook()

Aktualnie najlepsze cena zlecenia typu Buy: 30
+-----+-------+--------+-------+----------+
|  id | order |  type  | price | quantity |
+-----+-------+--------+-------+----------+
| 001 |  Buy  |  Add   |   20  |   100    |
| 002 |  Sell |  Add   |   25  |   200    |
| 003 |  Buy  |  Add   |   23  |    50    |
| 004 |  Buy  |  Add   |   23  |    70    |
| 003 |  Buy  | Remove |   23  |    50    |
| 005 |  Sell |  Add   |   28  |   100    |
| 006 |  Buy  |  Add   |   30  |    30    |
| 007 |  Sell |  Add   |   27  |    50    |
+-----+-------+--------+-------+----------+


### Removing existing order

Removing last order

In [10]:
order_book.remove("006")
order_book.print_orderbook()

+-----+-------+--------+-------+----------+
|  id | order |  type  | price | quantity |
+-----+-------+--------+-------+----------+
| 001 |  Buy  |  Add   |   20  |   100    |
| 002 |  Sell |  Add   |   25  |   200    |
| 003 |  Buy  |  Add   |   23  |    50    |
| 004 |  Buy  |  Add   |   23  |    70    |
| 003 |  Buy  | Remove |   23  |    50    |
| 005 |  Sell |  Add   |   28  |   100    |
| 006 |  Buy  |  Add   |   30  |    30    |
| 007 |  Sell |  Add   |   27  |    50    |
| 006 |  Buy  | Remove |   30  |    30    |
+-----+-------+--------+-------+----------+


Trying one more time (should not be allowed as it has already been withdrawn)

In [11]:
order_book.remove("006")
order_book.print_orderbook()

Id: 006 nie istnieje, lub zlecenie zostało już wycofane
+-----+-------+--------+-------+----------+
|  id | order |  type  | price | quantity |
+-----+-------+--------+-------+----------+
| 001 |  Buy  |  Add   |   20  |   100    |
| 002 |  Sell |  Add   |   25  |   200    |
| 003 |  Buy  |  Add   |   23  |    50    |
| 004 |  Buy  |  Add   |   23  |    70    |
| 003 |  Buy  | Remove |   23  |    50    |
| 005 |  Sell |  Add   |   28  |   100    |
| 006 |  Buy  |  Add   |   30  |    30    |
| 007 |  Sell |  Add   |   27  |    50    |
| 006 |  Buy  | Remove |   30  |    30    |
+-----+-------+--------+-------+----------+


trying add order of existing id (should not work because an order with the given id already exists)

In [12]:
order = Order(id="006", order_type=OrderType.Buy, operation_type=OperationType.Add, price=Decimal("24.00"), quantity=30)
order_book.add(order=order)
order_book.print_orderbook()

Zlecenie o id: 006 już istnieje
+-----+-------+--------+-------+----------+
|  id | order |  type  | price | quantity |
+-----+-------+--------+-------+----------+
| 001 |  Buy  |  Add   |   20  |   100    |
| 002 |  Sell |  Add   |   25  |   200    |
| 003 |  Buy  |  Add   |   23  |    50    |
| 004 |  Buy  |  Add   |   23  |    70    |
| 003 |  Buy  | Remove |   23  |    50    |
| 005 |  Sell |  Add   |   28  |   100    |
| 006 |  Buy  |  Add   |   30  |    30    |
| 007 |  Sell |  Add   |   27  |    50    |
| 006 |  Buy  | Remove |   30  |    30    |
+-----+-------+--------+-------+----------+


In [13]:
order = Order(id="001", order_type=OrderType.Buy, operation_type=OperationType.Add, price=Decimal("24.00"), quantity=30)
order_book.add(order=order)
order_book.print_orderbook()

Zlecenie o id: 001 już istnieje
+-----+-------+--------+-------+----------+
|  id | order |  type  | price | quantity |
+-----+-------+--------+-------+----------+
| 001 |  Buy  |  Add   |   20  |   100    |
| 002 |  Sell |  Add   |   25  |   200    |
| 003 |  Buy  |  Add   |   23  |    50    |
| 004 |  Buy  |  Add   |   23  |    70    |
| 003 |  Buy  | Remove |   23  |    50    |
| 005 |  Sell |  Add   |   28  |   100    |
| 006 |  Buy  |  Add   |   30  |    30    |
| 007 |  Sell |  Add   |   27  |    50    |
| 006 |  Buy  | Remove |   30  |    30    |
+-----+-------+--------+-------+----------+


checking if best price work (don't using removed orders)

In [14]:
order = Order(id="008", order_type=OrderType.Sell, operation_type=OperationType.Add, price=Decimal("22.00"), quantity=110)
order_book.add(order=order)
order_book.print_orderbook()

Aktualnie najlepsze cena zlecenia typu Buy: 23
+-----+-------+--------+-------+----------+
|  id | order |  type  | price | quantity |
+-----+-------+--------+-------+----------+
| 001 |  Buy  |  Add   |   20  |   100    |
| 002 |  Sell |  Add   |   25  |   200    |
| 003 |  Buy  |  Add   |   23  |    50    |
| 004 |  Buy  |  Add   |   23  |    70    |
| 003 |  Buy  | Remove |   23  |    50    |
| 005 |  Sell |  Add   |   28  |   100    |
| 006 |  Buy  |  Add   |   30  |    30    |
| 007 |  Sell |  Add   |   27  |    50    |
| 006 |  Buy  | Remove |   30  |    30    |
| 008 |  Sell |  Add   |   22  |   110    |
+-----+-------+--------+-------+----------+


In [15]:
order_book.close_connection()